<a href="https://colab.research.google.com/github/ssgalitsky/PedalNetRT/blob/master/colab_TPU_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



links:

PyTorch Lightning Documentation https://pytorch-lightning.readthedocs.io/en/stable/

In [1]:
#@title 1. clone  https://github.com/ssgalitsky/PedalNetRT.git
!rm -rf '/content/PedalNetRT'
import os
os.chdir('/content')
!git clone https://github.com/ssgalitsky/PedalNetRT.git


Cloning into 'PedalNetRT'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 227 (delta 0), reused 0 (delta 0), pack-reused 224
Receiving objects: 100% (227/227), 156.92 MiB | 27.54 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [2]:
#@title 2.1 install dependencies
!pip3 install torchvision
!pip3 install pytorch_lightning==0.9.0
use_tpu=False

     |████████████████████████████████| 409kB 2.3MB/s 
     |████████████████████████████████| 829kB 11.2MB/s 
     |████████████████████████████████| 2.8MB 10.6MB/s 
     |████████████████████████████████| 276kB 30.2MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=563704b6aa4c36eccaecb7f60e625e7273a5a10abbe44a2eaeca2b79694cc2e8
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=886d5362e41fd8e81dec5d5444a2ec3696f95aeda450dc4073cb3a19d6ec5f45
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully 

In [3]:
#@title 2.2 install optional dependencies (TPU)
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip3 install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl
use_tpu=True

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  65589      0 --:--:-- --:--:-- --:--:-- 65589
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 2.7MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][114.9 MiB/114.9 MiB]                                                
Operation completed over 1 objects/114.9 MiB.                                    
Copying gs://

In [4]:
import torch
use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()
use_tpu=True
#from google.colab import drive
#drive.mount('/content/drive')




In [13]:
#@title 3. prepare data
os.chdir('/content/PedalNetRT')
!python3 "prepare_data.py" "/content/PedalNetRT/data/s chain1.wav" "/content/PedalNetRT/data/s chain3.wav"


In [14]:
#@title 4. **train model** Beware, checkpoint files not visible via Colab webui (-rw-r--r--) 
import os
if use_tpu:
  assert os.environ['COLAB_TPU_ADDR']
os.chdir('/content/PedalNetRT')  
import pytorch_lightning
!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 --kernel_size=4 --dilation_depth=8
#!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 






2020-10-15 00:14:38.570211: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
GPU available: False, used: False
TPU available: True, using: 8 TPU cores
training on 8 TPU cores
INIT TPU local core: 0, global rank: 0 with XLA_USE_BF16=None
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)
INIT TPU local core: 2, global rank: 2 with XLA_USE_BF16=None
INIT TPU local core: 7, global rank: 7 with XLA_USE_BF16=None
INIT TPU local core: 1, global rank: 1 with XLA_USE_BF16=None
INIT TPU local core: 6, global rank: 6 with XLA_USE_BF16=None
INIT TPU local core: 4, global rank: 4 with XLA_USE_BF16=None
INIT TPU local core: 5, global rank: 5 with XLA_USE_BF16=None
INIT TPU local core: 3, global rank: 3 with XLA_USE_BF16=None

  |

In [15]:
!ls -d /content/PedalNetRT/lightning_logs/version_1/checkpoints/*

#!chmod -R 755 lightning_logs 


'/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt'
'/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.tmp_end.ckpt'


In [16]:
#@title 5. convert torch model (ckpt) to plugin model (json) 
# The .ckpt model must be converted to a .json model to run in the plugin. Usage:

#!python3 convert_pedalnet_to_wavnetva.py --model=your_trained_model.ckpt
!python3 convert_pedalnet_to_wavenetva.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt





2020-10-15 00:21:38.871550: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [17]:
!python test.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=99.ckpt

2020-10-15 00:21:56.102070: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [18]:
!python plot_wav.py

Error to signal (with pre-emphasis filter):  0.86500657
Error to signal (no pre-emphasis filter):  0.8650066
Creating spectrogram data..
plot_wav.py:116: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, frequencies, 10*np.log10(spectrogram))
<Figure size 1300x800 with 3 Axes>
<Figure size 1200x800 with 2 Axes>


In [ ]:
#@title 6. convert model from TPU to CPU format
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
